<a href="https://colab.research.google.com/github/amitkp57/Jupyter/blob/dev-muh/Jupyter-muhammad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Move to GPU mode if you are in Google Colab
Go to `Runtime` -> `Change runtime type` to activate GPU.

### Provide your credentials to the runtime


In [ ]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


### Mount Google Drive

In [ ]:
import os
from google.colab import drive
drive.mount('/content/gdrive')
WORKING_DIRECTORY = '/content/gdrive/MyDrive/Data/Jupyter/Jupyter'
os.environ['WORKING_DIRECTORY'] = WORKING_DIRECTORY
%cd $WORKING_DIRECTORY/
!ls -latr data/query_tables/

Mounted at /content/gdrive
/content/gdrive/MyDrive/Data/Jupyter/Jupyter
total 2596607
-rw------- 1 root root    230072 Mar 10 14:48 bigquery-public-data.covid19_aha.hospital_beds.npy
-rw------- 1 root root    407756 Mar 10 14:48 bigquery-public-data.covid19_aha.staffing.npy
-rw------- 1 root root   1980928 Mar 10 14:48 bigquery-public-data.covid19_ecdc.covid_19_geographic_distribution_worldwide.npy
-rw------- 1 root root   1980928 Mar 10 14:48 bigquery-public-data.covid19_ecdc_eu.covid_19_geographic_distribution_worldwide.npy
-rw------- 1 root root     39688 Mar 10 14:48 bigquery-public-data.covid19_geotab_mobility_impact.airport_traffic.npy
-rw------- 1 root root    317140 Mar 10 14:48 bigquery-public-data.covid19_geotab_mobility_impact.city_congestion.npy
-rw------- 1 root root   1287596 Mar 10 14:48 bigquery-public-data.covid19_geotab_mobility_impact.commercial_traffic.npy
-rw------- 1 root root    106972 Mar 10 14:48 bigquery-public-data.covid19_geotab_mobility_impact.commercial_tr

### Clone git repo

In [ ]:
%cd $WORKING_DIRECTORY
# !git clone https://github.com/amitkp57/Jupyter
!git pull
!pip install -r requirements.txt

/content/gdrive/MyDrive/Data/Jupyter/Jupyter
Already up to date.


### Setup meta data

Creates datasets.txt, tables.txt and columns.json in the /data folder. These files are used for querying Google Bigquery tables. 

In [ ]:
from scripts.MetaData import save_locally
DATA_PATH = f'{WORKING_DIRECTORY}/data'
save_locally(DATA_PATH)
print('Completed!')

## Test code

In [ ]:
# %cat $WORKING_DIRECTORY/scripts/QueryDatabase.py
# %load_ext autoreload
# %autoreload 2
import importlib
importlib.reload(qd)

<module 'scripts.QueryDatabase' from '/content/gdrive/My Drive/Data/Jupyter/Jupyter/scripts/QueryDatabase.py'>

In [ ]:
import scripts.QueryDatabase as qd
print(curr_cols)
test = qd.get_columns_values('bigquery-public-data.covid19_nyt.excess_deaths', curr_cols)
test


['month', 'week', 'deaths', 'expected_deaths', 'excess_deaths']


array([[1.000e+00, 2.000e+00, 3.000e+00, ..., 1.200e+01, 1.200e+01,
        1.200e+01],
       [0.000e+00, 0.000e+00, 0.000e+00, ..., 5.200e+01, 5.200e+01,
        5.200e+01],
       [4.538e+03, 4.244e+03, 4.312e+03, ..., 1.758e+03, 1.306e+03,
        9.683e+03],
       [0.000e+00, 0.000e+00, 0.000e+00, ..., 0.000e+00, 0.000e+00,
        0.000e+00],
       [0.000e+00, 0.000e+00, 0.000e+00, ..., 0.000e+00, 0.000e+00,
        0.000e+00]], dtype=float32)

In [ ]:
import scripts.QueryDatabase as qd
import numpy as np

exclude_types = ['STRING', 'GEOGRAPHY', 'DATE', 'DATETIME', 'TIMESTAMP']
all_cols = qd.get_columns_exclude(exclude_types)
columns = []
print('Column len: %d' % len(all_cols))

# Aggregate multiple columns with the same table
prev_cols = all_cols[0]
curr_cols = []
num_table = 0
for cols in all_cols:
  if (prev_cols['table'] == cols['table']):
    curr_cols.append(cols['column'])
  elif (len(curr_cols) > 0):
    num_table += 1
    print(prev_cols['table'])
    result = qd.get_columns_values(prev_cols['table'], curr_cols, 100000)
    with open(f'{os.environ["WORKING_DIRECTORY"]}/data/query_tables/' + prev_cols['table'] + '.npy', 'wb') as file:
      np.save(file, result)
    # for col_result in result:
    #   columns.append(col_result)
    curr_cols = [ cols['column'] ]
  prev_cols = cols
# Get the last columns
num_table += 1
print(prev_cols['table'])
result = qd.get_columns_values(prev_cols['table'], curr_cols)
with open(f'{os.environ["WORKING_DIRECTORY"]}/data/query_tables/' + prev_cols['table'] + '.npy', 'wb') as file:
  np.save(file, result)
# for col_result in result:
#   columns.append(col_result)

print('Total table query: %d' % num_table)
print('Column2 len: %d' % len(columns))


Column len: 8842
bigquery-public-data.covid19_aha.hospital_beds
bigquery-public-data.covid19_aha.staffing
bigquery-public-data.covid19_ecdc.covid_19_geographic_distribution_worldwide
bigquery-public-data.covid19_ecdc_eu.covid_19_geographic_distribution_worldwide
bigquery-public-data.covid19_geotab_mobility_impact.airport_traffic
bigquery-public-data.covid19_geotab_mobility_impact.city_congestion
bigquery-public-data.covid19_geotab_mobility_impact.commercial_traffic
bigquery-public-data.covid19_geotab_mobility_impact.commercial_traffic_by_industry
bigquery-public-data.covid19_geotab_mobility_impact.fuel_station_daily_fillups
bigquery-public-data.covid19_geotab_mobility_impact.fuel_station_weekly_fillups
bigquery-public-data.covid19_geotab_mobility_impact.port_traffic
bigquery-public-data.covid19_geotab_mobility_impact.us_border_volumes
bigquery-public-data.covid19_geotab_mobility_impact.us_border_wait_times
bigquery-public-data.covid19_geotab_mobility_impact_eu.airport_traffic
bigquery-

In [ ]:
import scripts.QueryDatabase as qd
import numpy as np

exclude_types = ['STRING', 'GEOGRAPHY', 'DATE', 'DATETIME', 'TIMESTAMP']
all_cols = qd.get_columns_exclude(exclude_types)
columns = []
print('Column len: %d' % len(all_cols))

# Aggregate multiple columns with the same table
prev_cols = all_cols[0]
curr_cols = []
col_th = 30
resume_table = 'bigquery-public-data.covid19_aha.hospital_beds'
cont_flag = False
is_single = True
suffix = 0
# resume
for cols in all_cols:
  if (not cont_flag):
    if (resume_table == cols['table']):
      cont_flag = True
      prev_cols = cols
    else:
      prev_cols = cols
      continue
  if (len(curr_cols) == col_th or prev_cols['table'] != cols['table']):
    print(prev_cols['table'] + ': ' + ','.join(curr_cols))
    result = qd.get_columns_values(prev_cols['table'], curr_cols, 30000)
    suff_str = ''
    if suffix > 0:
      suff_str += str(suffix)
    with open(f'{os.environ["WORKING_DIRECTORY"]}/data/query_tables/' + prev_cols['table'] + suff_str + '.npy', 'wb') as file:
      np.save(file, result)
    curr_cols = [ cols['column'] ]
    if prev_cols['table'] != cols['table']:
      if is_single:
        break
      suffix = 0
    else:
      suffix += 1
  else:
    curr_cols.append(cols['column'])
  prev_cols = cols
# Get the last columns
if not is_single:
  print(prev_cols['table'] + ': ' + ','.join(curr_cols))
  result = qd.get_columns_values(prev_cols['table'], curr_cols, 30000)
  suff_str = ''
  if suffix > 0:
    suff_str += str(suffix)
  with open(f'{os.environ["WORKING_DIRECTORY"]}/data/query_tables/' + prev_cols['table'] + suff_str + '.npy', 'wb') as file:
    np.save(file, result)

print('Column2 len: %d' % len(columns))

Column len: 8842
bigquery-public-data.covid19_aha.hospital_beds: cbsa_code,total_hospital_beds,acute_long_term_care_beds,alcohol_drug_abuse_dependency_inpatient_care_beds,burn_care_beds,cardiac_intensive_care_beds,gen_medical_surgical_adult_beds,gen_medical_surgical_pediatric_beds,intermediate_nursing_care_beds,medical_surgical_intensive_care_beds,neonatal_intensive_care_beds,neonatal_intermediate_care_beds,obstetric_care_beds,other_care_beds,other_intensive_care_beds,other_long_term_care_beds,other_special_care_beds,pediatric_intensive_care_beds,physical_rehabilitation_care_beds,psychiatric_care_beds,skilled_nursing_care_beds,num_airborne_infection_isolation_rooms
Column2 len: 0


In [ ]:
import scripts.QueryDatabase as qd
import numpy as np

exclude_types = ['STRING', 'GEOGRAPHY', 'DATE', 'DATETIME', 'TIMESTAMP']
all_cols = qd.get_columns_exclude(exclude_types)
curr_table = ''
for cols in all_cols:
  if (curr_table != cols['table']):
    curr_table = cols['table']
    print(curr_table)

bigquery-public-data.covid19_aha.hospital_beds
bigquery-public-data.covid19_aha.staffing
bigquery-public-data.covid19_ecdc.covid_19_geographic_distribution_worldwide
bigquery-public-data.covid19_ecdc_eu.covid_19_geographic_distribution_worldwide
bigquery-public-data.covid19_geotab_mobility_impact.airport_traffic
bigquery-public-data.covid19_geotab_mobility_impact.city_congestion
bigquery-public-data.covid19_geotab_mobility_impact.commercial_traffic
bigquery-public-data.covid19_geotab_mobility_impact.commercial_traffic_by_industry
bigquery-public-data.covid19_geotab_mobility_impact.fuel_station_daily_fillups
bigquery-public-data.covid19_geotab_mobility_impact.fuel_station_weekly_fillups
bigquery-public-data.covid19_geotab_mobility_impact.port_traffic
bigquery-public-data.covid19_geotab_mobility_impact.us_border_volumes
bigquery-public-data.covid19_geotab_mobility_impact.us_border_wait_times
bigquery-public-data.covid19_geotab_mobility_impact_eu.airport_traffic
bigquery-public-data.covid

In [ ]:
from os import listdir
from os.path import isfile, join
import numpy as np

datapath = f'{os.environ["WORKING_DIRECTORY"]}/data/query_tables/'
onlyfiles = [f for f in listdir(datapath) if isfile(join(datapath, f))]
columns = []
for file_name in onlyfiles:
  with open(f'{os.environ["WORKING_DIRECTORY"]}/data/query_tables/' + file_name, 'rb') as file:
    temp = np.load(file)
    for col in temp:
      columns.append(col)

print('Column2 len: %d' % len(columns))

Column2 len: 8842


In [ ]:
import numpy as np
import scipy.stats as stats

corr_list = []

for i in range(len(columns)):
  for j in range(i + 1, len(columns)):
    len_i = len(columns[i])
    len_j = len(columns[j])
    temp_i = columns[i]
    temp_j = columns[j]
    # print('len: %d, %d' % (len_i, len_j))
    if (len_i > len_j):
      np.random.shuffle(temp_i)
      temp_i = temp_i[:len_j]
    elif (len_j > len_i):
      np.random.shuffle(temp_j)
      temp_j = temp_j[:len_i]
    corr_list.append(stats.pearsonr(temp_i, temp_j))

corr_list

/usr/local/lib/python3.7/dist-packages/scipy/stats/stats.py:3508: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())


In [ ]:
import json
with open(f'{os.environ["WORKING_DIRECTORY"]}/data/pearson_corr.json', 'w') as file:
  json.dump(corr_list, file)

In [ ]:
import json
with open(f'{os.environ["WORKING_DIRECTORY"]}/data/pearson_corr.json') as file:
  test = json.load(file)

test

[[-0.006526964844085435, 0.7387673714516878],
 [0.0054184149723924, 0.7818988230877747],
 [0.02051332504289797, 0.2945470754391142],
 [0.01795143727301378, 0.3590024097377273],
 [-0.010361088243693199, 0.5965322719876853],
 [-0.012006415353817228, 0.5395679110079318],
 [-0.00523577061093342, 0.7890739461212976],
 [0.010506753040250129, 0.5913802771906213],
 [-0.015505687399839882, 0.42819712075869854],
 [-0.012343962216362685, 0.5282270584606461],
 [-0.008466811886964668, 0.6653031947310858],
 [-0.011975834372483923, 0.5406013544818378],
 [0.005643486901572685, 0.7730824571762809],
 [0.015542468615597615, 0.42710221242048935],
 [-0.011090418172045289, 0.5709446942628008],
 [-0.0036513646797459625, 0.8520056389821684],
 [-0.006757667071164244, 0.7298885545072841],
 [-0.01989914343563916, 0.3092444617719443],
 [-0.0008950279737798343, 0.9635256748773979],
 [-0.0005344374545992743, 0.978215637003314],
 [-0.0013989859567685096, 0.9430169365853387],
 [0.01954816127835654, 0.3178573434245278